In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scripts.helpers import extract_from_notation, Mode

LOCAL_DATA_PATH = "../data/raw/gun-violence-data_01-2013_03-2018.csv"
LOCAL_OUTPUT_PLOTS_PATH = "../documentation/plots/"

pd.set_option("display.max_columns", 100)
df = pd.read_csv(LOCAL_DATA_PATH)
df.shape
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [106]:
# Kolumny do zachowania
columns_to_keep = [
    "n_killed", "n_injured", "n_guns_involved",
    "gun_stolen", "gun_type", "incident_characteristics",
    "participant_age_group", "participant_gender",
    "participant_status", "participant_type"
]

# Zachowujemy tylko wybrane kolumny
df = df[columns_to_keep]

# Wyświetlamy nowy kształt i podgląd danych
print("Nowy rozmiar ramki danych:", df.shape)
df.head()

Nowy rozmiar ramki danych: (239677, 10)


,n_killed,n_injured,n_guns_involved,gun_stolen,gun_type,incident_characteristics,participant_age_group,participant_gender,participant_status,participant_type
0,0,4,NaN,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
1,1,3,NaN,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
2,1,3,2.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...
3,4,0,NaN,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
4,2,2,2.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...


In [107]:
# Funkcje pomocnicze
def categorize_killed(n):
    if pd.isna(n):
        return "Unknown killed"
    elif n == 0:
        return "No one killed"
    elif n <= 2:
        return "Few killed"
    else:
        return "Mass killing"

def categorize_injured(n):
    if pd.isna(n):
        return "Unknown injured"
    elif n == 0:
        return "No one injured"
    elif n <= 2:
        return "Few injured"
    else:
        return "Mass injury"

def categorize_guns(n):
    if pd.isna(n):
        return "Unknown guns"
    elif n == 1:
        return "Single gun"
    elif n == 2:
        return "Two guns"
    else:
        return "Multiple guns"

# Konwersja kolumn
df["n_killed"] = df["n_killed"].apply(categorize_killed)
df["n_injured"] = df["n_injured"].apply(categorize_injured)
df["n_guns_involved"] = df["n_guns_involved"].apply(categorize_guns)

# Podgląd po konwersji
df.head()


,n_killed,n_injured,n_guns_involved,gun_stolen,gun_type,incident_characteristics,participant_age_group,participant_gender,participant_status,participant_type
0,No one killed,Mass injury,Unknown guns,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
1,Few killed,Mass injury,Unknown guns,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
2,Few killed,Mass injury,Two guns,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...
3,Mass killing,No one injured,Unknown guns,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
4,Few killed,Few injured,Two guns,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...


In [108]:
import pandas as pd
import numpy as np

# ---------- Pomocnicze funkcje ----------

def extract_dict_from_notation(cell):
    """Zamienia 0::Value||1::Value na słownik {0: Value, 1: Value}"""
    if pd.isna(cell):
        return {}
    result = {}
    for item in str(cell).split("||"):
        if "::" in item:
            idx, val = item.split("::", 1)
            result[int(idx)] = val
    return result

def extract_from_notation(cell):
    """Zamienia 0::Value||1::Value na listę wartości [Value, Value]"""
    if pd.isna(cell):
        return []
    return [item.split("::")[1] for item in str(cell).split("||") if "::" in item]

def safe_concat_lists(*lists):
    """Łączy bezpiecznie listy stringów, ignorując błędne typy"""
    result = []
    for l in lists:
        if isinstance(l, list):
            result += [str(x) for x in l]
        elif pd.notna(l):
            result.append(str(l))
    return result

def combine_participant_attributes(row):
    combined = []

    types = row["participant_type_dict"]
    statuses = row["participant_status_dict"]
    genders = row["participant_gender_dict"]
    ages = row["participant_age_group_dict"]

    for idx, ptype in types.items():
        if idx in statuses:
            combined.append(f"{ptype} {statuses[idx]}")
        if idx in genders:
            combined.append(f"{ptype} {genders[idx]}")
        if idx in ages:
            combined.append(f"{ptype} {ages[idx]}")

    return combined

# ---------- Wczytanie danych ----------

# df = pd.read_csv(...) # już wczytane wcześniej

# ---------- Przetwarzanie pól notacji ----------

# Kolumny notacyjne jako listy
notated_columns = ["gun_stolen", "gun_type"]
for col in notated_columns:
    df[col] = df[col].apply(extract_from_notation)

# Kolumny numeryczne na listy stringów
for col in ["n_killed", "n_injured", "n_guns_involved"]:
    df[col] = df[col].apply(lambda x: [str(x)] if pd.notna(x) else [])

# incident_characteristics rozbijamy na listę (oddzielane ||)
df["incident_characteristics"] = df["incident_characteristics"].apply(
    lambda x: str(x).split("||") if pd.notna(x) else []
)

# participant_* jako słowniki
participant_cols = ["participant_type", "participant_status", "participant_gender", "participant_age_group"]
for col in participant_cols:
    df[col + "_dict"] = df[col].apply(extract_dict_from_notation)

# Generowanie cech łączonych
df["participant_combinations"] = df.apply(combine_participant_attributes, axis=1)

# ---------- Składanie pełnej transakcji ----------

df["transaction"] = df.apply(lambda row: safe_concat_lists(
    row["n_killed"],
    row["n_injured"],
    row["n_guns_involved"],
    row["gun_stolen"],
    row["gun_type"],
    row["incident_characteristics"],
    row["participant_combinations"]
), axis=1)

# Podgląd
df["transaction"].head()


0    [No one killed, Mass injury, Unknown guns, Sho...
1    [Few killed, Mass injury, Unknown guns, Shot -...
2    [Few killed, Mass injury, Two guns, Unknown, U...
3    [Mass killing, No one injured, Unknown guns, S...
4    [Few killed, Few injured, Two guns, Unknown, U...
Name: transaction, dtype: object

In [109]:
from mlxtend.preprocessing import TransactionEncoder

transactions = df["transaction"].tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_apriori = pd.DataFrame(te_ary, columns=te.columns_)

df_apriori.head(10)


,10mm,12 gauge,16 gauge,20 gauge,22 LR,223 Rem [AR-15],25 Auto,28 gauge,30-06 Spr,30-30 Win,300 Win,308 Win,32 Auto,357 Mag,38 Spl,380 Auto,40 SW,410 gauge,44 Mag,45 Auto,7.62 [AK-47],9mm,ATF/LE Confiscation/Raid/Arrest,ATF/LE Confiscation/Raid/Arrest|Possession (gun(s) found during commission of other crimes),ATF/LE Confiscation/Raid/Arrest|Possession (gun(s) found during commission of other crimes)|Gun(s) stolen from owner,Accidental Shooting,Accidental Shooting - Death,Accidental Shooting - Injury,Accidental Shooting at a Business,Accidental Shooting|Accidental Shooting - Death,Accidental Shooting|Accidental Shooting - Death|Accidental/Negligent Discharge,Accidental Shooting|Accidental Shooting - Death|Accidental/Negligent Discharge|Drug involvement,Accidental Shooting|Accidental Shooting - Death|Accidental/Negligent Discharge|Thought gun was unloaded,Accidental Shooting|Accidental Shooting - Injury,Accidental Shooting|Accidental Shooting - Injury|Accidental/Negligent Discharge,Accidental Shooting|Accidental Shooting - Injury|Brandishing/flourishing/open carry/lost/found,Accidental Shooting|Accidental Shooting - Injury|Cleaning gun,Accidental Shooting|Accidental Shooting - Injury|Drug involvement|Possession (gun(s) found during commission of other crimes),Accidental Shooting|Accidental/Negligent Discharge,"Accidental Shooting|Accidental/Negligent Discharge|Child Involved Incident|Child picked up & fired gun|Shots fired, no action (reported, no evidence found)",Accidental Shooting|Accidental/Negligent Discharge|Officer Involved Incident,Accidental Shooting|Accidental/Negligent Discharge|Possession (gun(s) found during commission of other crimes),Accidental Shooting|Hunting accident,Accidental/Negligent Discharge,Animal shot/killed,Animal shot/killed|Defensive Use,Armed robbery with injury/death and/or evidence of DGU found,Armed robbery with injury/death and/or evidence of DGU found|Brandishing/flourishing/open carry/lost/found,Armed robbery with injury/death and/or evidence of DGU found|Brandishing/flourishing/open carry/lost/found|Defensive Use,Armed robbery with injury/death and/or evidence of DGU found|Brandishing/flourishing/open carry/lost/found|Gun(s) stolen from owner,...,"Spree Shooting (multiple victims, multiple locations)",Stolen,Stolen/Illegally owned gun{s} recovered during arrest/warrant,Subject-Suspect Adult 18+,Subject-Suspect Arrested,Subject-Suspect Child 0-11,Subject-Suspect Female,Subject-Suspect Injured,"Subject-Suspect Injured, Arrested","Subject-Suspect Injured, Unharmed","Subject-Suspect Injured, Unharmed, Arrested",Subject-Suspect Killed,"Subject-Suspect Killed, Arrested","Subject-Suspect Killed, Injured","Subject-Suspect Killed, Unharmed","Subject-Suspect Killed, Unharmed, Arrested",Subject-Suspect Male,Subject-Suspect Teen 12-17,Subject-Suspect Unharmed,"Subject-Suspect Unharmed, Arrested",Suicide - Attempt,Suicide - Attempt|Possession (gun(s) found during commission of other crimes),Suicide^,TSA Action,Terrorism Involvement,Thought gun was unloaded,Two guns,Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator ),Unknown,Unknown guns,Unlawful purchase/sale,Victim Adult 18+,Victim Arrested,Victim Child 0-11,Victim Female,Victim Injured,"Victim Injured, Arrested","Victim Injured, Unharmed","Victim Injured, Unharmed, Arrested",Victim Killed,"Victim Killed, Arrested","Victim Killed, Injured","Victim Killed, Unharmed","Victim Killed, Unharmed, Arrested",Victim Male,"Victim Male, female",Victim Teen 12-17,Victim Unharmed,"Victim Unharmed, Arrested",Workplace shooting (disgruntled employee)
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,True,False,False,False,False,False,False

In [110]:
min_support = 0.01

col_frequencies = df_apriori.mean()

frequent_columns = col_frequencies[col_frequencies >= min_support].index.tolist()
df_apriori_reduced = df_apriori[frequent_columns]

df_apriori_reduced.columns.tolist()

selected_columns = [
    # Broń
    'Handgun', 'Rifle', 'Shotgun', 'Single gun', 'Multiple guns',

    # Sprawca
    'Subject-Suspect Adult 18+', 'Subject-Suspect Teen 12-17',
    'Subject-Suspect Killed', 'Subject-Suspect Arrested',

    # Ofiara
    'Victim Male', 'Victim Female', 'Victim Adult 18+', 'Victim Teen 12-17', 'Victim Killed',

    # Skutki
    'Few killed', 'No one killed', 'Victim Injured', 'Few injured', 'Mass injury',

    # Incydenty
    'Domestic Violence', 'Drug involvement', 'Gang involvement', 'Drive-by (car to street, car to car)',
    'Home Invasion', 'School Incident', 'Shot - Dead (murder, accidental, suicide)',
    'Shot - Wounded/Injured', 'Suicide^'
]

df_apriori_reduced = df_apriori_reduced.loc[:, ~df_apriori_reduced.columns.str.startswith('Victim')]

df_apriori_reduced = df_apriori_reduced.drop(columns=['Unknown', 'Unknown guns'])
df_apriori_reduced.columns.tolist()


['22 LR',
 '40 SW',
 '9mm',
 'ATF/LE Confiscation/Raid/Arrest',
 'Accidental Shooting',
 'Accidental Shooting - Injury',
 'Accidental/Negligent Discharge',
 'Armed robbery with injury/death and/or evidence of DGU found',
 'Bar/club incident - in or around establishment',
 'Brandishing/flourishing/open carry/lost/found',
 'Car-jacking',
 'Defensive Use',
 'Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator',
 'Defensive Use - Victim stops crime',
 'Domestic Violence',
 'Drive-by (car to street, car to car)',
 'Drug involvement',
 'Few injured',
 'Few killed',
 'Gang involvement',
 'Gun(s) stolen from owner',
 'Handgun',
 'Home Invasion',
 'Home Invasion - No death or injury',
 'Home Invasion - Resident injured',
 'Institution/Group/Business',
 'Kidnapping/abductions/hostage',
 'Mass injury',
 'Multiple guns',
 'Murder/Suicide',
 'No one injured',
 'No one killed',
 'Non-Aggression Incident',
 'Non-Shooting Incident',
 'Officer Involved Incident',
 'Officer Involved 

In [111]:
from mlxtend.frequent_patterns import apriori, association_rules

# filtered_df = df_apriori_reduced[df_apriori_reduced['Drug involvement'] == True]

frequent_itemsets = apriori(df_apriori_reduced,
                            min_support=0.04,
                            use_colnames=True,
                            verbose=1)

frequent_itemsets.sort_values(by="support", ascending=False).tail()
frequent_itemsets.shape

Processing 16 combinations | Sampling itemset size 875


(782, 2)

In [112]:
rules = association_rules(frequent_itemsets,
                          metric="confidence",
                          min_threshold=0.5)

# rules.sort_values(by="lift", ascending=False).head()
# rules[rules['confidence'] > 0.6].sort_values(by="confidence", ascending=True).head(20)

# Filter by multiple criteria
strong_rules = rules[
    (rules['lift'] >= 1.5) &
    (rules['confidence'] >= 0.7) &
    (rules['support'] >= 0.05)
].sort_values(['lift', 'confidence'], ascending=False)

strong_rules
# print(f"Strong rules: {strong_rules.shape[0]}")
# strong_rules.columns.tolist()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
4843,"(Subject-Suspect Adult 18+, Single gun, Few ki...","(No one injured, Subject-Suspect Male, Shot - ...",0.068079,0.098491,0.053347,0.783600,7.956065,1.0,0.046642,4.165937,0.938180,0.471165,0.759958,0.662621
4848,"(Shot - Dead (murder, accidental, suicide), Si...","(No one injured, Subject-Suspect Male, Few kil...",0.069564,0.097460,0.053347,0.766869,7.868520,1.0,0.046567,3.871377,0.938175,0.469280,0.741694,0.657119
4842,"(Subject-Suspect Male, Single gun, Few killed)","(No one injured, Shot - Dead (murder, accident...",0.071509,0.094915,0.053347,0.746018,7.859832,1.0,0.046560,3.563576,0.939988,0.471773,0.719383,0.654032
1333,"(Subject-Suspect Unharmed, Arrested, Shot - De...","(Subject-Suspect Male, Few killed)",0.054820,0.117825,0.050731,0.925413,7.854114,1.0,0.044272,11.827443,0.923293,0.416119,0.915451,0.677986
1334,"(Subject-Suspect Unharmed, Arrested, Few killed)","(Subject-Suspect Male, Shot - Dead (murder, ac...",0.054340,0.119461,0.050731,0.933584,7.814985,1.0,0.044239,13.257969,0.922150,0.412211,0.924574,0.679124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,(Subject-Suspect Unharmed),(Subject-Suspect Adult 18+),0.228762,0.476825,0.165172,0.722027,1.514238,1.0,0.056093,1.882105,0.440333,0.305640,0.468680,0.534213
1610,"(Subject-Suspect Male, Subject-Suspect Adult 1...",(No one injured),0.097732,0.594496,0.087952,0.899932,1.513773,1.0,0.029851,4.052273,0.376162,0.145549,0.753225,0.523938
3216,"(Subject-Suspect Male, Possession (gun(s) foun...",(No one injured),0.060761,0.594496,0.054607,0.898716,1.511728,1.0,0.018485,4.003631,0.360404,0.090913,0.750227,0.495285
1420,"(Subject-Suspect Male, No one killed, Possessi...",(No one injured),0.101712,0.594496,0.091239,0.897038,1.508906,1.0,0.030772,3.938398,0.375456,0.150817,0.746090,0.525256


In [113]:
# # Filter rules where 'Drug involvement' or 'Home invasion' is in the consequents
# drug_consequent_rules = strong_rules[
#     strong_rules['consequents'].apply(lambda x: 'Drug involvement' in x or 'Home invasion' in x or 'Gang involvement' in x)
# ]
#
# # print(f"Rules with 'Drug involvement' or 'Home invasion' as consequent: {drug_consequent_rules.shape[0]}")
# drug_consequent_rules